In [1]:
"""
You need to run this cell for the code in following cells to work.
"""

# Enable module reloading
%load_ext autoreload
%autoreload 2

import os
os.chdir("..")

import pandas as pd
from src.data.analysis import get_outliers
from src.data.pipelines import primary_use_pipeline, square_feet_pipeline, air_temperature_pipeline, \
    air_temperature_without_outliers_pipeline, dew_temperature_pipeline, \
    dew_temperature_without_outliers_pipeline, sea_level_pressure_pipeline, wind_speed_pipeline, \
    wind_speed_without_outliers_pipeline, wind_direction_pipeline, meter_pipeline
from src.data.feature_unions import buildings_fu, weather_fu, weather_without_outliers_fu

In [2]:
def check_mean_and_variance(df_column):
    mean = round(df_column.mean(), 2)
    var = round(df_column.var(), 2)
    print(f'mean = {mean}, variance = {var}')

# Data preprocessing

In [3]:
building_metadata = pd.read_csv('data/building_metadata.csv')
building_metadata

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN
...,...,...,...,...,...,...
1444,15,1444,Entertainment/public assembly,19619,1914.0,NaN
1445,15,1445,Education,4298,NaN,NaN
1446,15,1446,Entertainment/public assembly,11265,1997.0,NaN
1447,15,1447,Lodging/residential,29775,2001.0,NaN


In [4]:
bm_copy = building_metadata.copy()

## Buildings data

### Primary use

In this preprocessing we merge less numerous categories of primary use to category `Other` and encode values using One Hot Encoding.

In [5]:
primary_use_feature = pd.DataFrame(primary_use_pipeline.fit_transform(bm_copy))
primary_use_feature

,Education,Entertainment/public assembly,Lodging/residential,Office,Other,Public services
0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
1444,0.0,1.0,0.0,0.0,0.0,0.0
1445,1.0,0.0,0.0,0.0,0.0,0.0
1446,0.0,1.0,0.0,0.0,0.0,0.0
1447,0.0,0.0,1.0,0.0,0.0,0.0


We can see that there is only 6 categories (`Education`, `Entertainment/public assembly`, `Lodging/residential`, `Office`, `Other` and `Public services`) instead of 16 original and values are properly encoded.

### Square feet

In this preprocessing we scale values to zero mean unit variance.

In [6]:
square_feet_feature = pd.DataFrame(square_feet_pipeline.fit_transform(bm_copy))
square_feet_feature.head()

,square_feet
0,-0.764729
1,-0.807282
2,-0.783297
3,-0.617951
4,0.221212


In [7]:
check_mean_and_variance(square_feet_feature.square_feet)

mean = 0.0, variance = 1.0


We can see that are properly scaled.

### Union of features

The resulting buildings data looks following.

In [8]:
buildings_features = buildings_fu.union_features(building_metadata)
buildings_features

,site_id,building_id,Education,Entertainment/public assembly,Lodging/residential,Office,Other,Public services,square_feet
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.764729
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.807282
2,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.783297
3,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.617951
4,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.221212
...,...,...,...,...,...,...,...,...,...
1444,15.0,1444.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.654670
1445,15.0,1445.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.793032
1446,15.0,1446.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.730114
1447,15.0,1447.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.562953


## Weather data

In [9]:
train_weather = pd.read_csv('data/weather_train.csv')
train_weather

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6
...,...,...,...,...,...,...,...,...,...
139768,15,2016-12-31 19:00:00,3.0,NaN,-8.0,NaN,NaN,180.0,5.7
139769,15,2016-12-31 20:00:00,2.8,2.0,-8.9,NaN,1007.4,180.0,7.7
139770,15,2016-12-31 21:00:00,2.8,NaN,-7.2,NaN,1007.5,180.0,5.1
139771,15,2016-12-31 22:00:00,2.2,NaN,-6.7,NaN,1008.0,170.0,4.6


In [10]:
tw_copy = train_weather.copy()

### Air temperature

In this preprocessing we fill in missing values using rolling average and scale values to zero mean unit variance.

In [11]:
air_temperature_feature = pd.DataFrame(air_temperature_pipeline.fit_transform(tw_copy))
air_temperature_feature.head()

,air_temperature
0,0.995738
1,0.939274
2,0.788704
3,0.628723
4,0.525206


In [12]:
air_temperature_feature.air_temperature.isna().sum()

0

In [13]:
check_mean_and_variance(air_temperature_feature.air_temperature)

mean = 0.0, variance = 1.0


We can see that there is zero missing values and values are properly scaled.

In this preprocessing we focus on replacing outliers with 5th percentile or 95th percentile.

In [14]:
air_temperature_without_outliers_feature = \
    pd.DataFrame(air_temperature_without_outliers_pipeline.fit_transform(tw_copy))
air_temperature_without_outliers_feature.head()

,air_temperature
0,1.010217
1,0.952421
2,0.798300
3,0.634545
4,0.528587


In [15]:
get_outliers(air_temperature_without_outliers_feature, 'air_temperature')

lower bound: -2.87172198528123
upper bound: 2.9078396453263515


,air_temperature
61806,-2.871722
62355,-2.871722
62852,-2.871722
96766,-2.871722
97315,-2.871722
97812,-2.871722


We can see that outliers were correctly replaced (returned values are only rounding error).

### Dew temperature

In this preprocessing we fill in missing values using rolling average and scale values to zero mean unit variance.

In [16]:
dew_temperature_feature = pd.DataFrame(dew_temperature_pipeline.fit_transform(tw_copy))
dew_temperature_feature.head()

,dew_temperature
0,1.292475
1,1.404847
2,1.404847
3,1.353769
4,1.292475


In [17]:
dew_temperature_feature.dew_temperature.isna().sum()

0

In [18]:
check_mean_and_variance(dew_temperature_feature.dew_temperature)

mean = 0.0, variance = 1.0


We can see that there is zero missing values and values are properly scaled.

In this preprocessing we focus on replacing outliers with 5th percentile or 95th percentile.

In [19]:
dew_temperature_without_outliers_feature = \
    pd.DataFrame(dew_temperature_without_outliers_pipeline.fit_transform(tw_copy))
dew_temperature_without_outliers_feature.head()

,dew_temperature
0,1.314705
1,1.429847
2,1.429847
3,1.377510
4,1.314705


In [20]:
get_outliers(dew_temperature_without_outliers_feature, 'dew_temperature')

lower bound: -2.882740752309666
upper bound: 2.8952894356771677


,dew_temperature
62510,-2.882741
62793,-2.882741
97470,-2.882741
97753,-2.882741


We can see that outliers were correctly replaced.

### Sea level pressure

In this preprocessing we fill in missing values using rolling average and scale values to zero mean unit variance.

In [21]:
sea_level_pressure_feature = pd.DataFrame(sea_level_pressure_pipeline.fit_transform(tw_copy))
sea_level_pressure_feature.head()

,sea_level_pressure
0,0.466649
1,0.532188
2,0.532188
3,0.519080
4,0.505972


In [22]:
sea_level_pressure_feature.sea_level_pressure.isna().sum()

8755

Since there are still some missing values, we will look into them to find out why they were not filled in with some value.

In [23]:
train_weather[train_weather.sea_level_pressure.isna()].sea_level_pressure.isna().sum()

10618

We can see that although some values were filled in, most were not.

In [24]:
train_weather.groupby(['site_id']).count()

,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
site_id,,,,,,,,
0,8784,8781,4954,8781,8783,8699,8534,8784
1,8763,8762,1701,8762,0,8711,8760,8763
2,8783,8782,6429,8782,8717,8739,8192,8778
3,8780,8776,5138,8774,8743,8622,8627,8776
4,8783,8783,4553,8781,7466,8710,8678,8783
5,8755,8753,2725,8753,0,0,8460,8752
6,8782,8772,5790,8772,8782,8607,8003,8743
7,8614,8614,0,8591,731,8606,8614,8614
8,8784,8781,4954,8781,8783,8699,8534,8784


We can see that for 5th site there are no values of sea_level_pressure and therefore we have no way of filling in these data. Also precipitation is missing for that site.

We will probably throw away this attribute or site, because we have no way of filling in data.

In [25]:
check_mean_and_variance(sea_level_pressure_feature.sea_level_pressure)

mean = 0.0, variance = 1.0


### Wind speed

In this preprocessing we fill in missing values using rolling average and scale values to zero mean unit variance.

In [26]:
wind_speed_feature = pd.DataFrame(wind_speed_pipeline.fit_transform(tw_copy))
wind_speed_feature.head()

,wind_speed
0,-1.524278
1,-0.881804
2,-1.524278
3,-1.524278
4,-0.410656


In [27]:
wind_speed_feature.wind_speed.isna().sum()

0

In [28]:
check_mean_and_variance(wind_speed_feature.wind_speed)

mean = -0.0, variance = 1.0


We can see that there is zero missing values and values are properly scaled.

In this preprocessing we focus on replacing outliers with 5th percentile or 95th percentile.

In [29]:
wind_speed_without_outliers_feature = \
    pd.DataFrame(wind_speed_without_outliers_pipeline.fit_transform(tw_copy))
wind_speed_without_outliers_feature.head()

,wind_speed
0,-1.609404
1,-0.919681
2,-1.609404
3,-1.609404
4,-0.413884


In [30]:
get_outliers(wind_speed_without_outliers_feature, 'wind_speed')

lower bound: -2.6439894378040787
upper bound: 2.6898716490125114


,wind_speed


We can see that outliers were correctly replaced.

### Wind direction

In this preprocessing we fill in missing values using rolling average and scale values to zero mean unit variance.

In [31]:
wind_direction_feature = pd.DataFrame(wind_direction_pipeline.fit_transform(tw_copy))
wind_direction_feature.head()

,wind_direction
0,-1.615903
1,-0.983175
2,-1.615903
3,-1.615903
4,0.643840


In [32]:
wind_direction_feature.wind_direction.isna().sum()

0

In [33]:
check_mean_and_variance(wind_direction_feature.wind_direction)

mean = -0.0, variance = 1.0


We can see that there is zero missing values and values are properly scaled.

### Union of features

The resulting weather data looks following.

In [34]:
weather_features = weather_fu.union_features(train_weather)
weather_features

,site_id,timestamp,air_temperature,dew_temperature,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,0.995738,1.29247,-1.6159,-1.52428
1,0,2016-01-01 01:00:00,0.939274,1.40485,-0.983175,-0.881804
2,0,2016-01-01 02:00:00,0.788704,1.40485,-1.6159,-1.52428
3,0,2016-01-01 03:00:00,0.628723,1.35377,-1.6159,-1.52428
4,0,2016-01-01 04:00:00,0.525206,1.29247,0.64384,-0.410656
...,...,...,...,...,...,...
139768,15,2016-12-31 19:00:00,-1.0746,-1.56789,0.0111115,0.917124
139769,15,2016-12-31 20:00:00,-1.09343,-1.65983,0.0111115,1.77376
139770,15,2016-12-31 21:00:00,-1.09343,-1.48617,0.0111115,0.660134
139771,15,2016-12-31 22:00:00,-1.14989,-1.43509,-0.0792782,0.445976


In [35]:
weather_features = weather_without_outliers_fu.union_features(train_weather)
weather_features

,site_id,timestamp,air_temperature,dew_temperature,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,1.01022,1.31471,-1.6159,-1.6094
1,0,2016-01-01 01:00:00,0.952421,1.42985,-0.983175,-0.919681
2,0,2016-01-01 02:00:00,0.7983,1.42985,-1.6159,-1.6094
3,0,2016-01-01 03:00:00,0.634545,1.37751,-1.6159,-1.6094
4,0,2016-01-01 04:00:00,0.528587,1.31471,0.64384,-0.413884
...,...,...,...,...,...,...
139768,15,2016-12-31 19:00:00,-1.10896,-1.61618,0.0111115,1.01154
139769,15,2016-12-31 20:00:00,-1.12822,-1.71039,0.0111115,1.93118
139770,15,2016-12-31 21:00:00,-1.12822,-1.53244,0.0111115,0.735655
139771,15,2016-12-31 22:00:00,-1.18602,-1.4801,-0.0792782,0.505747


## Meter data

In [36]:
train_meter = pd.read_csv('data/train.csv')
train_meter

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.000
1,1,0,2016-01-01 00:00:00,0.000
2,2,0,2016-01-01 00:00:00,0.000
3,3,0,2016-01-01 00:00:00,0.000
4,4,0,2016-01-01 00:00:00,0.000
...,...,...,...,...
20216095,1444,0,2016-12-31 23:00:00,8.750
20216096,1445,0,2016-12-31 23:00:00,4.825
20216097,1446,0,2016-12-31 23:00:00,0.000
20216098,1447,0,2016-12-31 23:00:00,159.575


In [37]:
tm_copy = train_meter.copy()

### Meter reading

In this preprocessing we select only electricity meter type and meter readings smaller than 200. We also scale values to zero mean unit variance.

In [38]:
meter_reading_feature = pd.DataFrame(meter_pipeline.fit_transform(tm_copy))
meter_reading_feature.head()

,meter_reading
0,-1.074995
1,-1.074995
2,-1.074995
3,-1.074995
4,-1.074995


In [40]:
check_mean_and_variance(meter_reading_feature.meter_reading)

mean = -0.0, variance = 1.0


We can see that values are properly scaled. However we can not see that values are only for electricity meter type because returning this column would cause scaling of this electricity type as well.

### Union of features

The resulting meter data looks following.

TBA

## Data merging

TBA